In [0]:
%pip install databricks-sdk==0.57.0
dbutils.library.restartPython()

In [0]:
import os

from databricks.sdk import WorkspaceClient
from databricks.sdk.errors.platform import NotFound

# Name of the service principal created on install
# If you modified this field on install, change this to reflect the name
# you previously used.
# Alternatively, you may specify the exact ID of the service principal
# This can help disambiguate if there are multiple SPs with the same name
service_principal_name = "patyang-test-privatelink-sp"
service_principal_id = None

# Name of the catalog created on install
# Setting this to `None` will prevent the catalog from being deleted in
# the case you prefer keeping the data intact after uninstallation
catalog_name = "sec_lakehouse"

dbc = WorkspaceClient()


In [0]:
# Delete the previously created service principal
# We do this first to prevent the ASL control plane from continuing to make
# changes in the mean time while the rest of the uninstall occurs
if service_principal_id is None:
  sps = list(dbc.service_principals.list(
    filter=f"displayName eq '{service_principal_name}'"
  ))
  if len(sps) == 1:
    try:
      sp = sps[0]
      dbc.service_principals.delete(sp.id)
      print(f"Deleted service principal {sp.id} (name:{service_principal_name})")
    except NotFound:
      print(f"Skipping service principal deletion (name:{service_principal_name}) as it does not exist")
  elif len(sps) > 1:
    raise Exception(f"Multiple service principals named {service_principal_name} found, please specify `service_principal_id` to disambiguate")
  else:
    print(f"Skipping service principal deletion as could not find {service_principal_name}")

else:
  try:
    dbc.service_principals.delete(service_principal_id)
    print(f"Deleted service principal {sp.id}")
  except NotFound:
    print(f"Skipping service principal deletion (id:{service_principal_id}) as it does not exist")

In [0]:
if catalog_name is not None:
  # Force delete the catalog which includes all underlying schemas
  try:
    dbc.catalogs.delete(catalog_name, force=True)
  except NotFound:
    print(f"Skipping catalog deletion ({catalog_name}) as it does not exist")
else:
  print("Skipping catalog deletion as catalog_name is None")


In [0]:
# Check if the workspace has IP Access Lists in use and remove the DASL control plane allow list, if it exists
try:
    conf_value = dbc.workspace_conf.get_status(keys=["enableIpAccessLists"]).get("enableIpAccessLists", "false")
    ip_access_list_enabled = (False if conf_value is None or conf_value.lower() != "true" else True)
    if ip_access_list_enabled:
      print ("Workspace appears to use IP allow lists. Removing any previously created DASL IP Access List rules")
      for l in dbc.ip_access_lists.list():
        if l.enabled == True and l.label == "DASL" and l.list_type == settings.ListType.ALLOW:
          dbc.ip_access_lists.delete(l.list_id)
          print (f"Deleted IP allow list entry {l.list_id} (label:{l.label})")
    else:
        print ("Workspace does not appear to use IP allow lists")
except NotFound as e:
    print("This workspace does not have IP access lists, nothing to be done")

In [0]:
# The DASL uninstallation from your workspace is now complete. 
# Please contact Antimatter support to complete offboarding.
# Note that the service principals created during installation have been removed from the workspace
# but not fully deleted. You will need to manually delete them from the account console ('User Management' -> 'Service Principals')